## <span style="color:purple">ArcGIS API for Python</span>

<img src="img/Users_04_Analyst.png" style="width: 75%"></img>

## Demo: Analysis Using Drive Time Service Areas

In [1]:
from datetime import datetime
from IPython.display import HTML
import pandas as pd
import arcgis
import getpass

### Authenticate with Web GIS

In [2]:
portals_dict = {
    "esrifederal_gis": r"https://esrifederal.maps.arcgis.com",
    "natgov_gis": r"http://esri-natgov105.eastus.cloudapp.azure.com/arcgis",
    "dot_gis": r"http://dot.esri.com/portal",
    "dev_gis": r"http://govdev.eastus.cloudapp.azure.com/arcgis",
    "local_gis": r"https://anieto.esri.com/arcgis"
}

In [3]:
gis_url = portals_dict["esrifederal_gis"]

In [ ]:
if gis_url == portals_dict["esrifederal_gis"]:
    gis_app_id = r""
    print("Attempting to log in to '{0}'...".format(gis_url))
    gis = arcgis.gis.GIS(gis_url, client_id=gis_app_id)
    print("Successfully logged in as: " + gis.properties.user.username)
else:
    gis_username = getpass.getpass(prompt="Username: ")
    gis_pw = getpass.getpass(prompt="Password: ")
    print("Attempting to log in to '{0}'...".format(gis_url))
    gis = arcgis.gis.GIS(gis_url, gis_username, gis_pw, verify_cert=False)
    print("Successfully logged in as: " + gis.properties.user.username)

In [5]:
import arcgis.network as network

In [6]:
service_area_url = gis.properties.helperServices.serviceArea.url
service_area_url

'https://route.arcgis.com/arcgis/rest/services/World/ServiceAreas/NAServer/ServiceArea_World'

In [7]:
sa_layer = network.ServiceAreaLayer(service_area_url, gis=gis)

## Create the facilities layer

In [8]:
fire_station_1_coord = '139.546910,35.695729'
fire_station_2_coord = '139.673726,35.697988'

from arcgis.features import Feature, FeatureSet

f1 = Feature(geometry={'x':float(fire_station_1_coord.split(',')[0]),
                      'y':float(fire_station_1_coord.split(',')[1])})

f2 = Feature(geometry={'x':float(fire_station_2_coord.split(',')[0]),
                      'y':float(fire_station_2_coord.split(',')[1])})

fire_station_fset = FeatureSet([f1,f2], geometry_type='esriGeometryPoint', 
                            spatial_reference={'latestWkid': 4326})

fire_station_fset

<FeatureSet> 2 features

In [9]:
map1 = gis.map('Tokyo', zoomlevel=12)
map1

In [10]:
fire_truck_symbol = {"type":"esriPMS",
                     "url":"http://static.arcgis.com/images/Symbols/SafetyHealth/FireTruck.png",
                     "contentType": "image/png", "width":20, "height":20}

map1.draw(fire_station_fset, symbol=fire_truck_symbol)

## Compute service area

In [11]:
travel_modes = sa_layer.retrieve_travel_modes()
truck_mode = [t for t in travel_modes['supportedTravelModes'] if t['name'] == 'Trucking Time'][0]

result = sa_layer.solve_service_area(fire_station_fset, default_breaks=[5,10,15], 
                                     travel_direction='esriNATravelDirectionFromFacility',
                                     travel_mode=truck_mode)

In [12]:
result.keys()

dict_keys(['messages', 'saPolygons'])

In [13]:
result['saPolygons'].keys()

dict_keys(['fieldAliases', 'geometryType', 'spatialReference', 'features'])

In [14]:
poly_feat_list = []
for polygon_dict in result['saPolygons']['features']:
    f1 = Feature(polygon_dict['geometry'], polygon_dict['attributes'])
    poly_feat_list.append(f1)
poly_feat_list

[{"geometry": {"rings": [[[139.5585803990001, 35.740018845000066], [139.5579090120001, 35.73912239100002], [139.55813026400006, 35.73867225600003], [139.560152054, 35.738222122000025], [139.5603771210001, 35.73507881200004], [139.56149864200006, 35.73665046700006], [139.5630741120001, 35.73665046700006], [139.5648708340001, 35.735754013000076], [139.56599235500005, 35.73642540000003], [139.56936073300005, 35.734628677000046], [139.56981086700011, 35.733957291000024], [139.56958580000003, 35.733282089000056], [139.57115745500005, 35.73283195500005], [139.57115745500005, 35.732610703000034], [139.5716075900001, 35.731935501000066], [139.571382523, 35.73081398000005], [139.57250404400008, 35.73126030000003], [139.57385063200002, 35.730588913000076], [139.57452583300005, 35.72923851000007], [139.575647354, 35.72923851000007], [139.57632255600004, 35.72834205600003], [139.57879447900007, 35.72811698900006], [139.5792407990001, 35.72699546800004], [139.58238792400005, 35.72564506500004], [13

In [15]:
service_area_fset = FeatureSet(poly_feat_list, 
                         geometry_type=result['saPolygons']['geometryType'],
                         spatial_reference= result['saPolygons']['spatialReference'])

In [16]:
service_area_fset.df

,FacilityID,FromBreak,Name,ObjectID,Shape_Area,Shape_Length,ToBreak,SHAPE
0,1,10,Location 1 : 10 - 15,1,0.003145,0.821374,15,"{'rings': [[[139.5585803990001, 35.74001884500..."
1,1,5,Location 1 : 5 - 10,2,0.001552,0.453383,10,"{'rings': [[[139.56082725500005, 35.7242984770..."
2,1,0,Location 1 : 0 - 5,3,0.000512,0.120935,5,"{'rings': [[[139.55296516400006, 35.7081279750..."
3,2,10,Location 2 : 10 - 15,4,0.003695,1.111826,15,"{'rings': [[[139.70792579700003, 35.7575359340..."
4,2,5,Location 2 : 5 - 10,5,0.001934,0.467661,10,"{'rings': [[[139.6953487400001, 35.72385215800..."
5,2,0,Location 2 : 0 - 5,6,0.000606,0.136443,5,"{'rings': [[[139.6746883390001, 35.70947837800..."


## Visualize the service area on the map

In [17]:
colors = {5: [0, 128, 0, 90], 
          10: [255, 255, 0, 90], 
          15: [255, 0, 0, 90]}

fill_symbol = {"type": "esriSFS","style": "esriSFSSolid",
               "color": [115,76,0,255]}

In [18]:
for service_area in service_area_fset.features:
    
    #set color based on drive time
    fill_symbol['color'] = colors[service_area.attributes['ToBreak']]
    
    #set popup
    popup={"title": "Service area", 
            "content": "{} minutes".format(service_area.attributes['ToBreak'])}
    
    map1.draw(service_area.geometry, symbol=fill_symbol, popup=popup)

## Constructing service areas for different times of the day

In [19]:
times = [datetime(2017, 6, 10, h).timestamp() * 1000 
         for h in [6, 10, 14, 18, 22]]

# fire station location
fire_station = '-118.245847, 34.038608'

#loop through each time of the day and compute the service area
sa_results = []
for daytime in times:
    result = sa_layer.solve_service_area(facilities=fire_station, default_breaks=[5,10,15], 
                                         travel_direction='esriNATravelDirectionFromFacility',
                                         time_of_day=daytime, time_of_day_is_utc=False)
    sa_results.append(result)
sa_results

[{'messages': [],
  'saPolygons': {'features': [{'attributes': {'FacilityID': 1,
      'FromBreak': 10,
      'Name': 'Location 1 : 10 - 15',
      'ObjectID': 1,
      'Shape_Area': 0.012471792020728891,
      'Shape_Length': 3.138483944647704,
      'ToBreak': 15},
     'geometry': {'rings': [[[-118.24837303199996, 34.109071732000075],
        [-118.24859809899999, 34.103681564000055],
        [-118.24635124199995, 34.099416733000055],
        [-118.24365615799996, 34.09671974200006],
        [-118.24298286399994, 34.09739494300004],
        [-118.24275779699997, 34.09739494300004],
        [-118.24185943599997, 34.09739494300004],
        [-118.24141120899998, 34.09626960800006],
        [-118.24051284799998, 34.09671974200006],
        [-118.23983955399996, 34.09649467500003],
        [-118.23916625999999, 34.09514808700004],
        [-118.23714446999998, 34.094026566000025],
        [-118.23624610899998, 34.094026566000025],
        [-118.23579597499997, 34.092679977000046],
     

In [20]:
LA_fset_list=[]
for result in sa_results:
    poly_feat_list = []
    for polygon_dict in result['saPolygons']['features']:
        f1 = Feature(polygon_dict['geometry'], polygon_dict['attributes'])
        poly_feat_list.append(f1)
        
    service_area_fset = FeatureSet(poly_feat_list, 
                         geometry_type=result['saPolygons']['geometryType'],
                         spatial_reference= result['saPolygons']['spatialReference'])
    
    LA_fset_list.append(service_area_fset)
LA_fset_list    

[<FeatureSet> 3 features,
 <FeatureSet> 3 features,
 <FeatureSet> 3 features,
 <FeatureSet> 3 features,
 <FeatureSet> 3 features]

In [21]:
map2= gis.map("Los Angeles, CA")
map2

In [22]:
import time

animation_loops = 3

for loop in range(animation_loops):
    map2.clear_graphics()
    
    times = ['6 am', '10 am', '2 pm', '6 pm', '10 pm']
    j=0
    time.sleep(2)
    
    for fset in LA_fset_list:
        print(times[j])
        map2.draw(fset)
        j+=1
        time.sleep(2)

6 am
10 am
2 pm
6 pm
10 pm
6 am
10 am
2 pm
6 pm
10 pm
6 am
10 am
2 pm
6 pm
10 pm
